In [1]:
import yfinance as yf
from ssk import ssk
from score import get_scores_df
import pandas as pd
from stocksymbol import StockSymbol
from returns import get_monthly_adjusted_price

In [2]:
ssm = StockSymbol(ssk)

# Get tickers of companies listed in NASDAQ
symbol_list = ssm.get_symbol_list(market="US")
nasdaq_list = [x['symbol'] for x in symbol_list if x['exchange'] == 'NASDAQ']

# Randomly sample 100 of them
nasdaq_list = list(pd.Series(nasdaq_list).sample(100, random_state=1))

In [3]:
# Get the adjusted prices and scores

monthly_adjusted_price = get_monthly_adjusted_price(ticker_list=nasdaq_list, start_year = 2009)
monthly_adjusted_price.to_csv("monthly adjusted price.csv", index=False)

fundamental_scores = get_scores_df(ticker_list=nasdaq_list)
fundamental_scores.to_csv("fundamental scores.csv", index=False)

          CPLP
year          
2009  1.000000
2010  1.216509
2011  1.404927
2012  0.910473
2013  0.908259
2014  0.841575
2015  0.846965
2016  0.829709
2017  0.770155
2018  0.635641
2019  0.672602
2020  0.851520
2021  1.088543
2022  1.073167


In [4]:
scores = pd.read_csv("fundamental scores.csv")
adjusted_price = pd.read_csv("monthly adjusted price.csv")

In [5]:
adj_price_corr = adjusted_price.dropna(axis=1).corr()

In [6]:
from pairs import get_pair_distance
get_pair_distance(monthly_adjusted_price)

# monthly_adjusted_price[monthly_adjusted_price.index > (2010, 5)]

          AGEN      CPLP
AGEN  1.000000  0.203251
CPLP  0.203251  1.000000


CPLP  AGEN    0.203251
dtype: float64